# GenAI Workshop
## Lesson 2: Prompt Engineering

This lesson is intended to improve your prompt engineering skills.

During this lesson you will learn how to ...
* use a system prompt to define model behaviour
* extend system prompt to create workflows
* specify output format



### Set up the environment 

Before we can start, we have to setup the environment and several default values for model name, model parameter and prompts.  

In [ ]:
import os
from google import genai
from google.genai import types
import json
from pydantic import BaseModel
from enum import Enum
import time
from pathlib import Path
from google.colab import userdata


# Check runtime environment to make sure we are running in a colab environment. 
if os.getenv("COLAB_RELEASE_TAG"):
   COLAB = True
   print("Running on COLAB environment.") 
else:
   COLAB = False
   print("WARNING: Running on LOCAL environment.")

In [ ]:
# Clone the data repository into colab
!git clone https://github.com/openknowledge/workshop-genai-data.git
PROCESSED_DATA_PATH = "/content/workshop-genai-data/processed/gutenberg/"
BOOK_DB = PROCESSED_DATA_PATH + "books.db"

In [ ]:
# Initialize Google GenAI Client API with GOOGLE_API_KEY to be able to call the model.
# Note: GEMINI_API_KEY must be set as COLAB userdata before!
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

Initialize LOCAL environment


In [50]:
# Double check key settings by printing it out (or at least it length). 
if GOOGLE_API_KEY: 
    print(f' GOOGLE_API_KEY set with a length of {len(GOOGLE_API_KEY)}')
else: 
    print(f' ERROR: GOOGLE_API_KEY not set correctly!')

 GOOGLE_API_KEY set with a length of 39


### Definition of convenient functions  

The three following methods will simplify to work with the GEMINI genai model.
For details see function documentation.   

In [53]:
# Set default values for model, model parameters and prompt
DEFAULT_MODEL = "gemini-2.0-flash"
DEFAULT_CONFIG_TEMPERATURE = 0.9 
DEFAULT_CONFIG_TOP_K = 1
DEFAULT_CONFIG_MAX_OUTPUT_TOKENS = 200 
DEFAULT_SYSTEM_PROMPT = "Your are a friendly assistant"
DEFAULT_USER_PROMPT:str = " "

class MimeType(Enum):
    """
    Enum for MIME types.
    """
    JSON = "application/json"


class ResponseFormat(BaseModel):
    """
    Response format model for Gemini API.
    """
    response_mime_type: MimeType
    response_schema: type
    
history = [
    types.Part.from_bytes(
        data=Path(BOOK_DB).read_bytes(),
        mime_type='text/plain',
    )
]

def clear_history():
    """
    Clear the history of the conversation.
    """
    global history
    history = [
        types.Part.from_bytes(
            data=Path(BOOK_DB).read_bytes(),
            mime_type='text/plain',
        )
    ]

def generate_bookstore_bot_completion(
        user_prompt : str,
        response_schema: BaseModel | None = None,
        system_prompt: str = DEFAULT_SYSTEM_PROMPT,
        model_name: str = DEFAULT_MODEL,
        verbose: bool = False
        ): 
    """
    Call the GenAI model with function declarations and return the response.
    Args:
        user_prompt (str): The prompt to send to the model.
        response_format (ResponseFormat): The format of the response.
        system_prompt (str): The system prompt to use.
        model_name (str): The name of the model to use.
        verbose (bool): If True, print the response.
    Returns:
        str: The response from the model.
    """
    global history

    # Append file content if provided
    user_content = types.Content(
        role='user',
        parts=[types.Part.from_text(text=user_prompt)]
    )

    # Update history with user content
    history.append(user_content)

    # Configure response format
    response_mime_type = None
    if response_schema:
        response_mime_type = MimeType.JSON.value

    config = types.GenerateContentConfig(
        system_instruction=system_prompt,
        response_schema=response_schema,
        response_mime_type=response_mime_type,
    )

    # Send request with function declarations
    response = client.models.generate_content(
        model=model_name,
        contents=history,
        config=config,
    )

    # Update history with assistant content
    bot_content = types.ModelContent(response.text)

    history.append(user_content)
    history.append(bot_content)

    if verbose:
        print(f"User Prompt: {user_prompt}")
        print(f"Assistant Response: {response.text}")
    
    return response

In [54]:
def print_completion_result(completion_result, full:bool = False):
    
    """ Prints out the completion.    
    
    Parameters
    ----------
    completion_result : str
        A instance of GenerateContentResponse representing a completion 
    full : bool, optional [default: False]
    Whether to print all details of the completion or only the text. Defaults to False
    
    """            
    print(f'\nANSWER of genAI model: \n')
    if full:
        print(completion_result)
    else: 
        print(completion_result.text) 

In [55]:
def order_book(isbn:str):
    """ Orders a book by its ISBN number. 
    
    Parameters
    ----------
    isbn : str
        The ISBN number of the book to order.
    
    Returns 
    -------
    order_status : str
        The status of the order. 
    """
    # Simulate ordering the book
    print(f"Ordering book with ISBN: {isbn}")
    # Print "." every 0.5 seconds
    for _ in range(5):
        print(".", end="", flush=True)
        time.sleep(0.5)
    print()
    if isbn == "978-3-51593-12345-6":
        print("Success: You ordered 'A Study in Scarlet'!")
        print("You completed this exercise successfully!")
    else:
        print("Error: Unknown ISBN number.")

### Important notice: History
In this (and only this) exercise, we implemented a history feature, so that the conversation can keep on going. If you mess up, use the following function to clear the history.

In [56]:
# Clean the chat history
clear_history()

### Exercise 01: Basic interaction
Your task is to create a simple chatbot, which is able to order a book based on the description of a book.  
Imagine the following situation:
A customer of a bookstore wants to buy a book, but does not remember its title.  
The customer might ask an employee: *"I'm looking for this book, where Sherlock Holmes and Watson meet the first time."*
In response an employee will use his/her extensive knowledge of books and say: "That's "A Study in Scarlet"! Should I place an order for this book?"
  
The employee's reaction described here, should now be done by a chatbot. Your task is to provide a system prompt for this bot. 
The bot should:  
* Respond to the customer by naming the book title, year of publication and a short summary of the book.
* In addition the bot should ask, if the book should be ordered.  

**Hints**: The creators of the bot (aka teacher of this workshop) already provided knowledge about books to the bot. You do not need to worry about this.


 

In [57]:
# Do not change the user prompt.
user_prompt = "I'm looking for this book, where Sherlock Holmes and Watson meet the first time."

# TODO: Define the respective system prompt.
system_prompt = "You are a friendly assistant working at a bookstore." \
    "Your task is to help customers to order the book they are looking for." \
    "If a customer does not know the title of the book, use your extensive knowledge to find information" \
    "Provide the title of the book, the year of publication and a short summary of the book" \
    "In addition, ask the customer if you should order the book for them."

# We provide some knowledge to the model about the book contents.
response = generate_bookstore_bot_completion(user_prompt=user_prompt,system_prompt=system_prompt)
print_completion_result(response)


ANSWER of genAI model: 

Ah, you're looking for the book that introduces the world to Sherlock Holmes and Dr. Watson! That would be "A Study in Scarlet" by Arthur Conan Doyle. It was published in 1995.

In this story, Dr. Watson, a veteran, returns to London and becomes roommates with the brilliant and eccentric Sherlock Holmes. They are soon embroiled in a baffling murder case involving a victim found with the word "RACHE" scrawled in blood. Holmes's deductive genius is on full display as he unravels a tale of revenge that stretches from the American West to the heart of London.

Would you like me to order "A Study in Scarlet" for you?



### Exercise 02: Extend the process
Extend the system prompt even further.
After the bot provided the information about the book and asks if it should be ordered, the *customer might say*: "Yes, please!" As an result, the bot will order the book. This process is finished by the bot replying with an object like {"isbn": "978-4-23050-12345-6"}. This object represents the payload for ordering the book using an API.
Your task is to extend the system prompt in order to achieve this behaviour.

In [58]:
# TODO: Define the respective system prompt.
system_prompt = "You are a friendly assistant working at a bookstore." \
    "Your task is to help customers to order the book they are looking for." \
    "If a customer does not know the title of the book, use your extensive knowledge to find information" \
    "Provide the title of the book, the year of publication and a short summary of the book" \
    "In addition, ask the customer if you should order the book for them." \
    "If the customers wants to order a book, you reply with the following template: 'isbn = *isbn-of-the-book-to-order*'"


In [ ]:
# Clear the chat history
clear_history()

# The customer is looking for a book.
customer_initial_prompt = "I'm looking for this book, where Sherlock Holmes and Watson meet the first time."
print(f'Customer:\n {customer_initial_prompt}\n')

# The bookstore bot answers the customer.
response = generate_bookstore_bot_completion(user_prompt=customer_initial_prompt, system_prompt=system_prompt)
print(f'Bookstore bot:\n {response.text}\n')

# The customer wants to order the book.
customer_answer = "Yes, I like to order the book."
print(f'Customer:\n {customer_answer}\n')

# The bookstore bot answers the customer.
response = generate_bookstore_bot_completion(user_prompt=customer_answer, system_prompt=system_prompt)
print(f'Bookstore bot:\n {response.text}\n')

Customer:
 I'm looking for this book, where Sherlock Holmes and Watson meet the first time.

Bookstore bot:
 The book you are looking for is titled "A Study in Scarlet" written by Arthur Conan Doyle, published in 1995. It is the first novel featuring Sherlock Holmes and Dr. John Watson, detailing their initial meeting and their first case together in Victorian London, involving a mysterious murder with connections to the American West.

Would you like me to order it for you?


Customer:
 Yes I like to order the book.

Bookstore bot:
 isbn = 978-3-51593-12345-6




In [79]:
# TODO: Order the book by using the ISBN number provided by the bot.
isbn = "978-3-51593-12345-6"
order_book(isbn=isbn)

Ordering book with ISBN: 978-3-51593-12345-6
.....
Success: You ordered 'A Study in Scarlet'!
You completed this exercise successfully!


### Exercise 03: Give choices
After the initial customer request, the bot should give the user another choice. Instead of ordering the book, the bot can provide an url to the ebook version of the book. Update the system prompt in order to achieve this.

In [ ]:
# TODO Update the system prompt to include the option to provide an url to the ebook instead of ordering the book.
system_prompt = """
You are a friendly and knowledgeable assistant working at a bookstore. Your primary task is to help customers find and order the books they are looking for, even if they are unsure of the exact title.
Instructions & Context:
1. Finding the Book:
- If the customer provides the title, locate the book and confirm the details.
- If the customer is unsure of the title, use your extensive knowledge to help them identify it based on any information they provide (e.g., author, plot, genre).
2. Provide Essential Information:
- For each book you identify, provide the following details:
  - Title of the Book
  - Year of Publication
  - Short Summary of the Book
3. Offer Two Clear Choices:
- Choice 1: You order the book for the customer.
- Choice 2: You provide a link to the ebook.
- Formulate these choices in a clear and engaging way, such as: Would you like me to order the physical copy for you, or would you prefer a link to the ebook version?
4. Handling the Customer's Choice:
- If the customer wants the ebook link, provide the URL directly. Use your knowledge to find the ebook link.
- If the customer wants to order the book, respond with the following template: isbn = isbn-of-the-book-to-order

Examples:
Customer: I’m looking for a book about a young wizard who goes to a magical school.
Assistant: I believe you are referring to Harry Potter and the Sorcerer's Stone, published in 1997. It's the first book in the Harry Potter series, where a young boy discovers he's a wizard and attends Hogwarts School of Witchcraft and Wizardry.
Would you like me to order the physical copy for you, or would you prefer a link to the ebook version?

Customer Response: I’d like the ebook, please.
Assistant: Here is the link to the ebook: [ebook-link]

Customer: I want to order The Catcher in the Rye.
Assistant: The Catcher in the Rye, published in 1951, is a classic novel that follows the journey of Holden Caulfield as he navigates teenage angst and alienation in New York City.
Would you like me to order the physical copy for you, or would you prefer a link to the ebook version?

Customer Response: I want to order it.
Assistant: isbn = 9780316769488"
"""

In [62]:
# Clear the chat history
clear_history()

# The customer is looking for a book.
customer_initial_prompt = "I'm looking for this book, where Sherlock Holmes and Watson meet the first time."
print(f'Customer:\n {customer_initial_prompt}\n')

# The bookstore bot answers the customer.
response = generate_bookstore_bot_completion(user_prompt=customer_initial_prompt, system_prompt=system_prompt)
print(f'Bookstore bot:\n {response.text}\n')

# The customer wants to to have the link to the ebook.
customer_answer = "I love ebooks. Why should I order a book, if I get an ebook for free. Please provide the link!"
print(f'Customer:\n {customer_answer}\n')

# The bookstore bot answers the customer.
response = generate_bookstore_bot_completion(user_prompt=customer_answer, system_prompt=system_prompt)
print(f'Bookstore bot:\n {response.text}\n')

Customer:
 I'm looking for this book, where Sherlock Holmes and Watson meet the first time.

Bookstore bot:
 The book you're looking for is "A Study in Scarlet" by Arthur Conan Doyle. It was published in 1995. The story begins with Watson returning from military service and moving in with Holmes. Holmes is called upon to investigate the mysterious murder of Enoch Drebber.

Would you like me to order the book for you, or would you prefer a link to the ebook?


Customer:
 I love ebooks. Why should I order a book, if I get an ebook for free. Please provide the link!

Bookstore bot:
 Here is the link to the ebook: https://www.gutenberg.org/ebooks/244




In [63]:
# Test if the old process of ordering a book still works

# Clear the chat history
clear_history()

# The customer is looking for a book.
customer_initial_prompt = "I'm looking for this book, where Sherlock Holmes and Watson meet the first time."
print(f'Customer:\n {customer_initial_prompt}\n')

# The bookstore bot answers the customer.
response = generate_bookstore_bot_completion(user_prompt=customer_initial_prompt, system_prompt=system_prompt)
print(f'Bookstore bot:\n {response.text}\n')

# The customer wants to order the book.
customer_answer = "Please order the book. I love spending money for stuff I can get for free!"
print(f'Customer:\n {customer_answer}\n')

# The bookstore bot answers the customer.
response = generate_bookstore_bot_completion(user_prompt=customer_answer, system_prompt=system_prompt)
print(f'Bookstore bot:\n {response.text}\n')

Customer:
 I'm looking for this book, where Sherlock Holmes and Watson meet the first time.

Bookstore bot:
 The book you're looking for is "A Study in Scarlet" by Arthur Conan Doyle, published in 1995. This novel marks the debut of Sherlock Holmes and Dr. John Watson, who become roommates and solve a perplexing murder case together in Victorian London.

Would you like me to order the book for you, or would you prefer a link to the ebook?


Customer:
 Please order the book. I love spending money for stuff I can get for free!

Bookstore bot:
 isbn = 978-3-51593-12345-6




### Additional Information


#### Response Formatting
Defining the output format within the prompt (like we did when specifing how to return the placed order) is not suitable for real world usecases. Parsing the response of the genAI model can lead to parsing errors or worse, like getting transported to the core of the application and create unexpected behaviour. Therefore we should prescribe the output format using the sdk and validate the response.

In [ ]:
# Define a schema for the response
class BookOrder(BaseModel):
    """
    Model for book order.
    """
    isbn: str
    title: str

# Create a more simple example for ordering a book.
system_prompt = "You are a friendly assistant working at a bookstore." \
    "Your task is to help customers to order the book they are looking for."
user_prompt = "Please order the book 'Study in Scarlett'!"

# Clear the chat history
clear_history()
response = generate_bookstore_bot_completion(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    response_schema=BookOrder,
)

# Validate the response. This will raise an error if the response does not match the schema.
BookOrder(**json.loads(response.text))

BookOrder(isbn='978-3-51593-12345-6', title='A Study in Scarlet')

#### Function Calling
In a real world scenario, the bot would not return the order-object to the user, but to a dedicated API. Below you see an example of how to automatically call an external tool (i.e. an api for ordering a book) using function calling.

##### Convenient functions

In [73]:
# Define classes and convenient functions
class FunctionCall(BaseModel):
    """
    Function call model for Gemini API.
    """
    name: str
    arguments: dict

class FunctionCallResponse(BaseModel):
    """
    Function call response model for Gemini API.
    """
    name: str
    result: dict

class GeminiResponse(BaseModel):
    """
    Gemini response model.
    """
    text: str | None
    function_call: FunctionCall | None


class MimeType(Enum):
    """
    Enum for MIME types.
    """
    JSON = "application/json"


class ResponseFormat(BaseModel):
    """
    Response format model for Gemini API.
    """
    response_mime_type: MimeType
    response_schema: type
    

# Call the gemini model with the capability of calling functions and providing function call results
def generate_gemini_completion(
        user_prompt : str | None = None,
        response_format: ResponseFormat | None = None,
        system_prompt: str = DEFAULT_SYSTEM_PROMPT,
        model_name: str = DEFAULT_MODEL, 
        function_declarations: list | None = None,
        function_call_response: FunctionCallResponse | None = None,
        verbose: bool = False
        ): 
    """
    Call the GenAI model with function declarations and return the response.
    Args:
        user_prompt (str): The prompt to send to the model.
        response_format (ResponseFormat): The format of the response.
        system_prompt (str): The system prompt to use.
        model_name (str): The name of the model to use.
        function_declarations (list): List of function declarations.
        verbose (bool): If True, print the response.
    Returns:
        GeminiResponse: The response from the model.
    """
    global history

    contents = history.copy()

    # Add user_prompt to contents
    if user_prompt:
        # Append user prompt to history
        user_content = types.Content(
            role='user',
            parts=[types.Part.from_text(text=user_prompt)]
        )
        contents.append(user_content)

    # Add function call response to contents
    if function_call_response:
        function_response_part = types.Part.from_function_response(
            name=function_call_response.name,
            response={"result": function_call_response.result},
        )
        contents.append(types.Content(role="user", parts=[function_response_part]))

    # Configure tools
    tools = []
    if function_declarations:
        tools.append(types.Tool(function_declarations=function_declarations))

    # Configure response format
    response_schema = None
    response_mime_type = None
    if response_format:
        response_schema = response_format.response_schema
        response_mime_type = response_format.response_mime_type.value

    config = types.GenerateContentConfig(
        tools=tools,
        system_instruction=system_prompt,
        response_schema=response_schema,
        response_mime_type=response_mime_type,
    )

    # Send request with function declarations
    response = client.models.generate_content(
        model=model_name,
        contents=contents,
        config=config,
    )

    function_call = None
    if response.candidates[0].content.parts[0].function_call:
        function_call_gemini = response.candidates[0].content.parts[0].function_call
        function_call = FunctionCall(
            name=function_call_gemini.name,
            arguments=function_call_gemini.args,
        )
        # Append the function call to the history
        contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call_gemini)]))

    # Append the model response to the history
    model_response = response.candidates[0].content.parts[0].text
    if model_response:
            model_content = types.ModelContent(model_response)
            contents.append(model_content)
    
    # Update the history
    history += contents
    if verbose:
        print(f"History: {history}")
    
    return GeminiResponse(
        text=model_response,
        function_call=function_call,
    )

In [ ]:
# Function to handle function calls
def handle_function_call(response: GeminiResponse, user_prompt: str, system_prompt: str, function_declarations: list[dict], function_map: dict) -> tuple[str, GeminiResponse]:
    """
    Handles function calls from the Gemini model response. If a function call is detected,
    it calls the specified function with the provided arguments and sends back the result.

    Args:
        response: The response object from the Gemini model.
        user_prompt: The user's original prompt.
        system_prompt: The system's initial prompt.
        function_declarations: List of function declarations for Gemini to reference.
        function_map: A dictionary mapping function names to actual Python functions.

    Returns:
        A tuple of (response_text, final_response), where:
        - response_text is the plain text response for the user.
        - final_response is the complete response object from Gemini.
    """
    if response.function_call:
        func_name = response.function_call.name
        func_args = response.function_call.arguments
        
        if func_name in function_map:
            # Call the function dynamically
            result = function_map[func_name](**func_args)
            
            # Prepare the response for the Gemini model
            function_call_response = FunctionCallResponse(
                name=func_name,
                result=result,
            )
            final_response = generate_gemini_completion(
                user_prompt=user_prompt,
                system_prompt=system_prompt,
                function_declarations=function_declarations,
                function_call_response=function_call_response,
            )
            return final_response.text, final_response
        
        else:
            raise ValueError(f"Function call not recognized: {func_name}")
    
    # If no function call, just return the text
    return response.text, response

##### Providing the tool
In this section we define the available tools for the models. In order to provide tool capabilities to the model, we need function declarations (which is just text) and the actual implementation. The actual implementation is what we use to call the function ourself, if the models decides to do so.

In [66]:
# Actual implementation of the function to be called. The responsibility to call the function is on our side (not gemini)
def call_book_order_api(isbn: str):
    """
    Call the book order API with the given ISBN number.
    Args:
        isbn (str): The ISBN number of the book to order.
    Returns:
        dict: The response from the API.
    """

    if isbn == "978-3-51593-12345-6":
        return {"status": "success", "order_id": "123-FUNCTION-CALLING-456"}
    else:
        return {"status": "error", "message": "Invalid ISBN number."}


In [67]:
# Declaration of the available function

# Name of the function to be called. We need this later to map the declaration to the implementation.
order_api_function_name = "order_book"

# Function declaration for the book order API. This is what the model will see.
call_book_order_api_declaration = {
    "name": order_api_function_name,
    "description": "Order a book, which is provided by its ISBN number.",
    "parameters": {
        "type": "object",
        "properties": {
            "isbn": {
                "type": "string",
                "description": "The ISBN number of the book.",
            }
        },
        "required": ["isbn"],
    },
}

# Create a list of function declarations
function_declarations = [call_book_order_api_declaration]

In [68]:
# Mapping function names to actual functions. We use this for dynamic function calling.
function_map = {
    order_api_function_name: call_book_order_api,
}

In [69]:
# The updated system prompt for the bookstore bot. We do not include information about the response format.
# We tell the model to use tools.

system_prompt = "You are a friendly assistant working at a bookstore." \
    "Your task is to help customers to order the book they are looking for." \
    "If a customer does not know the title of the book, use your extensive knowledge to find information" \
    "Provide the title of the book, the year of publication and a short summary of the book" \
    "In addition, you should give two choices to the customer." \
    "Choice 1: You order the book for the customer." \
    "Choice 2: You provide a link to the ebook." \
    "Formulate the choices in a sentence." \
    "If the customer wants to provide a link to the ebook, you reply with the url of the ebook." \
    "If the customer wants to order a book,  you use tools to order a book." \
    "Use your extensive knowledge to find the isbn number of the book." \
    "Do not provide the isbn number in the response."

##### Interactive chat

In [ ]:
# An interactive chat with the bookstore bot to show the dynamic function calling capabilities.
def start_chat():
  
  # Clear the chat history
  clear_history()

  print("Type 'exit' to stop the chat.\n")
  while True:
      # 🗣️ Get user input
      user_prompt = input("You: ")
      
      # 🚪 Exit condition
      if user_prompt.lower() == "exit":
          print("Exiting chat. Goodbye!")
          break

      # 🔄 Generate response from the model
      response = generate_gemini_completion(
          user_prompt=user_prompt,
          system_prompt=system_prompt,
          function_declarations=function_declarations,
      )

      # 🤖 Handle potential function calls
      response_text, _ = handle_function_call(
          response,
          user_prompt=user_prompt,
          system_prompt=system_prompt,
          function_declarations=function_declarations,
          function_map=function_map,
      )
      
      # 💬 Display the bot's response
      print(f"\033[1mYou:\033[0m {user_prompt}\n")
      print(f"\033[1mBookstore Bot:\033[0m {response_text}\n")


In [ ]:
# Let's chat!
# You can use this prompt to start a conversation with the bot: I'm looking for this book, where Sherlock Holmes and Watson meet the first time.
start_chat()